# XGBoost Forecasting Model (Model A - Seasonality Focus)

This notebook implements XGBoost (Extreme Gradient Boosting) for time series forecasting with engineered temporal features.

## XGBoost Advantages
- **Non-linear Patterns**: Captures complex relationships
- **Feature Importance**: Identifies key drivers
- **Robust**: Handles missing values and outliers
- **Flexible**: Can incorporate many features

## Configuration
- **Features**: Temporal (month, quarter, week), lag features [1,3,6,12], rolling statistics
- **Hyperparameters**: n_estimators=200, max_depth=6, learning_rate=0.05
- **Validation**: Time series split (no data leakage)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## Section 1: Load Time Series Data

In [ ]:
# Load company-level time series
data_path = Path('../data/processed/monthly_aggregated_full_company.parquet')

if not data_path.exists():
    data_path = Path('../data/processed/monthly_aggregated_full_company.csv')
    df = pd.read_csv(data_path)
    df['date'] = pd.to_datetime(df['date'])
else:
    df = pd.read_parquet(data_path)

df = df.sort_values('date').reset_index(drop=True)

print(f"Loaded: {len(df)} months ({df['date'].min()} to {df['date'].max()})")

## Section 2: Feature Engineering

Create temporal features and lag features for XGBoost.

In [ ]:
target_metrics = [
    'total_orders',
    'total_km_billed',
    'total_km_actual',
    'total_tours',
    'total_drivers',
    'revenue_total',
    'external_drivers',
    'vehicle_km_cost',      # NEW: KM-based transportation cost
    'vehicle_time_cost',    # NEW: Time-based transportation cost
    'total_vehicle_cost'    # NEW: Total vehicle operational cost
]

# Backward compatibility check
if 'total_km' in df.columns and 'total_km_billed' not in df.columns:
    target_metrics = [m.replace('total_km_billed', 'total_km') if m == 'total_km_billed' else m for m in target_metrics]
    target_metrics = [m for m in target_metrics if m != 'total_km_actual']  # Remove if not available

In [ ]:
# Split data: Use all 36 months as training (no validation split needed for baselines)
# Since we don't have 2025 data yet, we'll use the full historic data as training
# and demonstrate forecasting forward

print("Creating train/validation split...")
print("="*80)

# For Prophet models, we'll use last 6 months as validation
split_date = '2024-07-01'

train_df = df[df['date'] < split_date].copy()
val_df = df[df['date'] >= split_date].copy()

print(f"\nTraining data:")
print(f"  Date range: {train_df['date'].min()} to {train_df['date'].max()}")
print(f"  Months: {len(train_df)}")

print(f"\nValidation data:")
print(f"  Date range: {val_df['date'].min()} to {val_df['date'].max()}")
print(f"  Months: {len(val_df)}")

print(f"\n✓ Split complete!")
print(f"  Training: {len(train_df)} months (Jan 2022 - Jun 2024)")
print(f"  Validation: {len(val_df)} months (Jul 2024 - Dec 2024)")

In [ ]:
def create_features(df, target_col):
    """
    Create temporal and lag features for XGBoost.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Time series dataframe with 'date' column
    target_col : str
        Name of the target column to create lag features for
    
    Returns:
    --------
    pd.DataFrame
        Dataframe with engineered features
    """
    df_feat = df.copy()
    
    # Temporal features
    df_feat['year'] = df_feat['date'].dt.year
    df_feat['month'] = df_feat['date'].dt.month
    df_feat['quarter'] = df_feat['date'].dt.quarter
    df_feat['week'] = df_feat['date'].dt.isocalendar().week
    df_feat['day_of_year'] = df_feat['date'].dt.dayofyear
    df_feat['weekday'] = df_feat['date'].dt.weekday
    
    # Lag features (previous months' values)
    df_feat[f'lag_1'] = df_feat[target_col].shift(1)
    df_feat[f'lag_3'] = df_feat[target_col].shift(3)
    df_feat[f'lag_6'] = df_feat[target_col].shift(6)
    df_feat[f'lag_12'] = df_feat[target_col].shift(12)
    
    # Rolling statistics
    df_feat[f'rolling_mean_3'] = df_feat[target_col].rolling(window=3, min_periods=1).mean()
    df_feat[f'rolling_std_3'] = df_feat[target_col].rolling(window=3, min_periods=1).std()
    df_feat[f'rolling_mean_6'] = df_feat[target_col].rolling(window=6, min_periods=1).mean()
    
    # Growth rate features
    df_feat[f'growth_rate_1'] = df_feat[target_col].pct_change(1)
    df_feat[f'growth_rate_3'] = df_feat[target_col].pct_change(3)
    df_feat[f'growth_rate_12'] = df_feat[target_col].pct_change(12)
    
    return df_feat

print("✓ Feature engineering function defined")

## Section 3: Train/Validation Split

In [ ]:
# Split data
train_end = '2024-06-30'
val_start = '2024-07-01'
val_end = '2024-12-31'

print(f"Data Split:")
print(f"  Training: up to {train_end}")
print(f"  Validation: {val_start} to {val_end}")

## Section 4: Train XGBoost Models

Train one XGBoost model per target metric.

In [ ]:
exclude_cols = [
    target_col, 'date', 'year_month',
    'total_orders', 'total_km_billed', 'total_km_actual', 'total_tours',
    'total_drivers', 'external_drivers', 'internal_drivers', 'revenue_total',
    'vehicle_km_cost', 'vehicle_time_cost', 'total_vehicle_cost',  # NEW: Cost metrics
    'total_km',  # For backward compatibility
    'Delivery', 'Leergut', 'Pickup/Multi-leg', 'Retoure/Abholung', 'km_per_order'
]

## Section 5: Feature Importance Analysis

Identify which features contribute most to predictions.

In [ ]:
# Feature importance for first metric
metric = target_metrics[0]
model = xgb_models[metric]
feature_cols = xgb_feature_cols[metric]

# Get feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 Features for {metric}:")
print("="*50)
print(importance_df.head(10).to_string(index=False))

# Plot feature importance
fig = go.Figure([
    go.Bar(
        y=importance_df.head(15)['feature'],
        x=importance_df.head(15)['importance'],
        orientation='h'
    )
])

fig.update_layout(
    title=f"Top 15 Feature Importances - {metric}",
    xaxis_title="Importance",
    yaxis_title="Feature",
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)

fig.show()

# Save
results_dir = Path('../results')
results_dir.mkdir(exist_ok=True)
fig.write_html(results_dir / f'xgboost_feature_importance_{metric}.html')
print(f"\n✓ Saved: results/xgboost_feature_importance_{metric}.html")

## Section 6: Generate Forecasts

Generate predictions for validation and future periods.

In [ ]:
def generate_xgb_forecasts(model, df_feat, feature_cols, target_col, val_start, val_end, horizon=18):
    """
    Generate XGBoost forecasts.
    
    Note: For true future forecasts, we'd need to implement recursive forecasting
    (using predictions as lag features). For now, we'll forecast validation period
    where actual lag values are available.
    """
    # Validation period
    val_df = df_feat[(df_feat['date'] >= val_start) & (df_feat['date'] <= val_end)].copy()
    
    if len(val_df) == 0:
        print(f"  ⚠️  No validation data available for {target_col}")
        return np.array([]), np.array([])
    
    # Get features (handle any NaN)
    X_val = val_df[feature_cols]
    
    # Check for NaN in features
    if X_val.isna().any().any():
        print(f"  ⚠️  Warning: NaN values in validation features, filling with mean")
        X_val = X_val.fillna(X_val.mean())
    
    # Predict
    predictions = model.predict(X_val)
    
    return predictions, val_df['date'].values

# Generate forecasts
xgb_forecasts = {}
xgb_forecast_dates = {}

for metric in target_metrics:
    model = xgb_models[metric]
    df_feat = xgb_dataframes[metric]
    feature_cols = xgb_feature_cols[metric]
    
    predictions, dates = generate_xgb_forecasts(
        model, df_feat, feature_cols, metric, val_start, val_end
    )
    
    xgb_forecasts[metric] = predictions
    xgb_forecast_dates[metric] = dates
    
    print(f"\n{metric}:")
    print(f"  Validation forecast: {len(predictions)} months")

## Section 7: Model Evaluation

In [ ]:
def calculate_metrics(y_true, y_pred, model_name, metric_name):
    """Calculate forecast accuracy metrics."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    return {
        'model': model_name,
        'metric': metric_name,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    }

# Calculate metrics
results = []

val_df = df[(df['date'] >= val_start) & (df['date'] <= val_end)]

for metric in target_metrics:
    if len(xgb_forecasts[metric]) > 0:
        y_true = val_df[metric].values
        y_pred = xgb_forecasts[metric]
        
        metrics = calculate_metrics(y_true, y_pred, 'XGBoost', metric)
        results.append(metrics)

results_df = pd.DataFrame(results)

print("\nXGBoost Model Performance (Validation Period):")
print("="*80)
print(results_df.to_string(index=False))

# Compare with previous models
try:
    baseline_df = pd.read_csv('../data/processed/baseline_metrics.csv')
    prophet_df = pd.read_csv('../data/processed/prophet_metrics.csv')
    sarimax_df = pd.read_csv('../data/processed/sarimax_metrics.csv')
    
    print("\n" + "="*80)
    print("Model Comparison (MAPE):")
    print("="*80)
    
    for metric in target_metrics:
        print(f"\n{metric}:")
        
        if len(results_df[results_df['metric'] == metric]) > 0:
            xgb_mape = results_df[results_df['metric'] == metric]['MAPE'].values[0]
            prophet_mape = prophet_df[prophet_df['metric'] == metric]['MAPE'].values[0]
            sarimax_mape = sarimax_df[sarimax_df['metric'] == metric]['MAPE'].values[0]
            baseline_best_mape = baseline_df[baseline_df['metric'] == metric]['MAPE'].min()
            
            print(f"  XGBoost: {xgb_mape:.2f}%")
            print(f"  SARIMAX: {sarimax_mape:.2f}%")
            print(f"  Prophet: {prophet_mape:.2f}%")
            print(f"  Best Baseline: {baseline_best_mape:.2f}%")
            
            best_model = min([
                ('XGBoost', xgb_mape),
                ('SARIMAX', sarimax_mape),
                ('Prophet', prophet_mape)
            ], key=lambda x: x[1])
            
            print(f"  → Best: {best_model[0]} ({best_model[1]:.2f}%)")
except Exception as e:
    print(f"\n⚠️  Could not load previous model metrics: {e}")

## Section 8: Forecast Visualization

In [ ]:
# Visualize forecasts
train_df = df[df['date'] <= train_end]
val_df = df[(df['date'] >= val_start) & (df['date'] <= val_end)]

for metric in target_metrics:
    if len(xgb_forecasts[metric]) == 0:
        continue
        
    fig = go.Figure()
    
    # Historical training data
    fig.add_trace(
        go.Scatter(
            x=train_df['date'],
            y=train_df[metric],
            mode='lines+markers',
            name='Historical (Training)',
            line=dict(color='black', width=2)
        )
    )
    
    # Actual validation values
    if len(val_df) > 0:
        fig.add_trace(
            go.Scatter(
                x=val_df['date'],
                y=val_df[metric],
                mode='lines+markers',
                name='Actual (Validation)',
                line=dict(color='green', width=3)
            )
        )
    
    # XGBoost forecast
    fig.add_trace(
        go.Scatter(
            x=xgb_forecast_dates[metric],
            y=xgb_forecasts[metric],
            mode='lines+markers',
            name='XGBoost Forecast',
            line=dict(color='purple', width=2, dash='dash')
        )
    )
    
    fig.update_layout(
        title=f"XGBoost Forecast - {metric.replace('_', ' ').title()}",
        xaxis_title="Date",
        yaxis_title=metric.replace('_', ' ').title(),
        height=600,
        hovermode='x unified'
    )
    
    fig.show()
    
    # Save
    fig.write_html(results_dir / f'xgboost_forecast_{metric}.html')
    print(f"\n✓ Saved: results/xgboost_forecast_{metric}.html")

## Section 9: Save Results

In [ ]:
# Save performance metrics
output_dir = Path('../data/processed')
results_df.to_csv(output_dir / 'xgboost_metrics.csv', index=False)
print(f"✓ Saved metrics: data/processed/xgboost_metrics.csv")

# Save validation forecasts
if len(val_df) > 0 and len(results_df) > 0:
    forecast_output = pd.DataFrame({
        'date': val_df['date'],
        'year_month': val_df['year_month'].astype(str) if 'year_month' in val_df.columns else val_df['date'].dt.to_period('M').astype(str)
    })
    
    for metric in target_metrics:
        if len(xgb_forecasts[metric]) > 0:
            forecast_output[metric] = xgb_forecasts[metric]
    
    forecast_output.to_csv(output_dir / 'xgboost_forecast_validation.csv', index=False)
    print(f"✓ Saved validation forecasts: data/processed/xgboost_forecast_validation.csv")

print(f"\n{'='*80}")
print(f"XGBOOST MODEL COMPLETE!")
print(f"{'='*80}")
print(f"\nKey Findings:")
for metric in target_metrics:
    if len(results_df[results_df['metric'] == metric]) > 0:
        mape = results_df[results_df['metric'] == metric]['MAPE'].values[0]
        print(f"  • {metric}: MAPE = {mape:.2f}%")
print(f"\nNext: Run notebook 13 for Weighted Prophet model")